Let's import the necessary libraries and files to get started

In [94]:
import numpy as np
import pandas as pd

import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
bucket='ceo-turnover-data'
data_key = 'pre_processed_v4_CEO.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

data = pd.read_csv(data_location)

data is a pandas dataframe storing our preprocessed dataset. Let's print out data.head() to check it imported correctly and see how it's structured

In [97]:
print(data.head())

   Age                                       Company Name   Director Name  \
0   73  COSTCO WHOLESALE CORP (Costco Companies Inc pr...     Jim Sinegal   
1   28            Morris & Garritano Insurance Agency Inc  Brendan Morris   
2   29  Madison Industries Inc (Madison Capital Partne...   Larry Gies Jr   
3   33                              Crowley Maritime Corp  Tom Crowley Jr   
4   33                     Enterprise Solutions Group Inc     Savas Karas   

   Number of Records               Role Name           Seniority  \
0                  1           President/CEO  Executive Director   
1                  1                     CEO  Executive Director   
2                  1  Chairman/President/CEO  Executive Director   
3                  1  Chairman/President/CEO  Executive Director   
4                  1           President/CEO  Executive Director   

   Tenure (Years) Turnover (YES/NO)  Year  
0               7                NO  2000  
1               7                NO  200

Now we're cooking with gas! Let's trim our data to only include predictors and labels.

In [98]:
data = data[['Age', 'Tenure (Years)', 'Turnover (YES/NO)']]

print(data.head())
print("\n We have {} rows of {} columns".format(data.shape[0], data.shape[1]))

   Age  Tenure (Years) Turnover (YES/NO)
0   73               7                NO
1   28               7                NO
2   29               7                NO
3   33               7                NO
4   33               7                NO

 We have 291294 rows of 3 columns


One-hot encode our label data and store it in a separate array

In [99]:
pre_y = np.array(data[['Turnover (YES/NO)']])
y = np.zeros((data.shape[0], 2))
for i in range(data.shape[0]):
    if pre_y[i] == 'NO':
        y[i, 0] = 1
    else:
        y[i, 1] = 1
        
        
y1 = np.zeros((data.shape[0]))


Aaand let's also put our predictor data in separate array

In [107]:
X = np.array(data[['Age', 'Tenure (Years)']])

print(type(X))

avg_age = np.sum(X[:,0]) / data.shape[0]

print("The average CEO is {} years old".format(round(avg_age)))

<class 'numpy.ndarray'>
The average CEO is 58.0 years old


Now time to import sagemaker and instantiate the model

In [109]:
import sagemaker

sess = sagemaker.Session()

import io
import sagemaker.amazon.common as smac

buf = io.BytesIO()
# smac.write_numpy_to_dense_tensor(buf, X, y)

from sagemaker.tensorflow import TensorFlow as tf
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import LinearSVC

train_X, test_X, train_y, test_y = train_test_split(X, y)

In [111]:
model = LinearSVC()

print(train_y.shape)

model.fit(train_X, train_y)

(218470, 2)


ValueError: bad input shape (218470, 2)

In [69]:
pred_y = model.predict(test_X)

model.score(test_X, test_y)

0.042277620130521165

In [73]:
sum = 0

print(max(pred_y))

for idx, y in enumerate(pred_y):
#     print(int(y + 0.5))
    if y == test_y[idx]:
        sum += 1
        
print(sum)

[0.30659351]
0
